### ***PRE-ELABORAZIONE DATI***

*Nel caso di FastAI la pre-elaborazione è fatta automaticamente dal classificatore; l'unica cosa da fare in modo da rendere fruibile la rete è quella di suddividere il dataset in una parte per il train e una per il test e rendere numerica la variabile target.*

In [ ]:
dep_var = # Inserire nome della variabile target del dataset 

In [ ]:
from sklearn.model_selection import train_test_split

# train 50% e test 50%
train, test = train_test_split(dataset, test_size=0.50)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder delle due variabili target, quindi da stringhe a interi sia in test che in train e salvo i valori nelle variabili y_test a y_train
# y_test viene utilizzata per la valutazione del modello

y_train = LabelEncoder().fit_transform(train[dep_var])
y_test = LabelEncoder().fit_transform(test[dep_var])


# sostituisco i valori originari con la codifica precedentemente fatta
target_index = train.columns.get_loc(dep_var)
train.iloc[:, target_index] = y_train

# elimino la colonna relativa al target nel dataset di test poiché non utile
test = test.drop(dep_var, axis=1)

In [ ]:
# Importazione dataset dal drive 
import pandas as pd
from collections import Counter

path = # /*Inserire percorso del file*/
dataset = pd.read_csv(path)

In [ ]:
from fastai import *
from fastai.tabular import *
from fastai.tabular.data import *

### ***MODEL***

In [ ]:
""" Definisco delle variabili utili al nostro modello per saper come trattare i dati. 
Distinguendo variabile target, continue e categoriali. Inoltre in procs vengono passati degli 
ottimizzatori che vengono applicati ai dati prima del training: 
FillMissing vengono sostituiti eventuali valori NaN, Categorify si occupa di 
codificare le variabili categoriali e Normalize si  occupa di normalizzare i valori continui."""

cat_names = # array nome variabile categoriali (se non presenti array vuoto)
cont_names = [col for col in train.columns if col not in cat_names and col != dep_var]
procs = [FillMissing, Categorify, Normalize]

In [ ]:
# Splitto il dataset di train in una parte di validation
from sklearn.model_selection import train_test_split

test_size = # dimensione in percentuale rispetto a tutto il dataset.

def SplitSet(df):
     train, valid = train_test_split(df, test_size = test_size)
     split_val = len(train)
     train = train.append(valid)
     return train, split_val

traindf, idx = SplitSet(train)

In [ ]:
# Lista da passare al nostro tabular learner per effettuare la fase di test
test = TabularList.from_df(test, cat_names= cat_names, cont_names=cont_names, procs=procs)

# Lista contenete gli insiemi di training e validation
data = TabularList.from_df(traindf, cat_names= cat_names, cont_names=cont_names, procs=procs)
data = data.split_by_idx(list(range(idx, len(traindf))))
data = data.label_from_df(cols=dep_var)
data = data.add_test(test, label= 0)
data = data.databunch()

In [ ]:
# Creazione del nostro modello con due Hidden layers da, rispettivamente, 200 e 100 neuroni.
# Per valutare la performance durante l'addestramento viene usata l'accuratezza

learn = tabular_learner(data, layers=[200,100], 
                        metrics=accuracy, emb_drop=0.01, callback_fns=ShowGraph)

In [ ]:
# Visualizzazione struttura del modello
learn

### ***TRAINING***

In [ ]:
"""Troviamo il valore di learning rate da utilizzare per l'algoritmo di discesa del gradiente. 
  Prendiamo un valore mediano rispetto al punto di max pendenza """
learn.lr_find()
learn.recorder.plot()

In [ ]:
# Addestramento del modello con lr massimo = 1e-02 e wd = 0.02
learn.fit_one_cycle(2, 1e-02, wd=0.02)

### ***PREDICTION***

In [ ]:
# Valutazione del dataset di test ottenuto splittando il dataset di train
import numpy as np

pred, *_ = learn.get_preds(DatasetType.Test)
label = np.argmax(pred, 1)

y_pred = np.array(label)

### ***EVALUATION***

In [ ]:
# Matrice di confusione, accuracy, classification_report
from sklearn.metrics import *

# y_test è la variabile che contiene i valori effettivi
# y_pred contiene i valori predetti dal modello
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

acc = accuracy_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
# non presente nella libreria, calcolo mediante formula
f2 = (1+2**2)*((precision*recall)/((2**2*precision)+recall))

In [ ]:
# Multilabel confusione Matrix
mcm = multilabel_confusion_matrix(y_test, y_pred)
print(mcm)

In [ ]:
# Confusione matrix
print(cm)

In [ ]:
# True/False Positive and True/False Negative
FP = cm.sum (axis = 0) - np.diag (cm) 
FN = cm.sum (axis = 1) - np.diag (cm) 
TP = np.diag (cm) 
TN = cm.sum () - (FP + FN + TP)

print('True positive: ', TP)
print('True negative: ', TN)
print('False positive: ', FP)
print('False negative: ', FN)

FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)

print('True positive rate: ', TPR)
print('True negative rate: ', TNR)
print('False positive rate: ', FPR)
print('False negative rate: ', FNR)

In [ ]:
# Stampa report con gli indice della performance
print(report)

In [ ]:
# Stampa di tutte le varie metriche
print('Accuracy: ', acc)
print('Precision_weighted: ', precision)
print('Recall_weighted: ', recall)
print('mcc: ', mcc)
print('f2: ', f2)